**Creación de la base de datos**

Esta base de datos se construyó mediante la integración de dos fuentes principales de información cinematográfica: el dataset "Full TMDB Movies Dataset 2024 (1M Movies)" de Kaggle, que contiene 1 000 000 películas, y el dataset "Top 10000 Popular Movies Dataset" de la misma plataforma. La fusión se realizó utilizando `pd.merge` en Python, tomando como llave el título original de las películas (`original_title`) y aplicando una unión interna para conservar únicamente los registros presentes en ambos conjuntos. Durante este proceso, se añadieron automáticamente sufijos a las columnas duplicadas (`_1M` y `_10k`) para identificar su origen, y posteriormente se seleccionó un subconjunto relevante de variables relacionadas con características técnicas, desempeño comercial y recepción crítica de las producciones.

In [2]:
import pandas as pd
import numpy as np

In [3]:
# se empleó el sitio "HugginFace" para poder subir los archviso originales de Kaggle

url_one_million = "https://huggingface.co/datasets/cataxmonger7/one_million/resolve/main/one_million.csv"
url_ten_thousand = "https://huggingface.co/datasets/cataxmonger7/ten_thousand/resolve/main/ten_thousand.csv"

# cargar 

one_million = pd.read_csv(
    url_one_million, 
    on_bad_lines='skip',  
    engine='python',     
    encoding='utf-8'     
)

ten_thousand = pd.read_csv(
    url_ten_thousand,
    on_bad_lines='skip',
    engine='python',
    encoding='utf-8'
)

In [4]:
one_million.columns #evaluar columnas

Index(['id', 'title', 'vote_average', 'vote_count', 'status', 'release_date',
       'revenue', 'runtime', 'adult', 'backdrop_path', 'budget', 'homepage',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'tagline', 'genres',
       'production_companies', 'production_countries', 'spoken_languages',
       'keywords'],
      dtype='object')

In [5]:
ten_thousand.columns #evaluar columnas

Index(['Unnamed: 0', 'id', 'original_language', 'original_title', 'popularity',
       'release_date', 'vote_average', 'vote_count', 'genre', 'overview',
       'revenue', 'runtime', 'tagline'],
      dtype='object')

In [6]:
ten_thousand = ten_thousand.drop(columns=["Unnamed: 0"]) #eliminar columna sin nombre

In [7]:
ten_thousand.columns #verificar cambio

Index(['id', 'original_language', 'original_title', 'popularity',
       'release_date', 'vote_average', 'vote_count', 'genre', 'overview',
       'revenue', 'runtime', 'tagline'],
      dtype='object')

In [8]:
#combiar las bases de datos
merged = pd.merge(
    one_million,
    ten_thousand,
    on='id',
    how='inner', #para solo dejar casos comunes
    suffixes=('_1M', '_10k')
)

# todas las columnas disponibles
print("Todas las columnas después del merge:")
for i, col in enumerate(merged.columns):
    print(f"{i}: {col}")

# buscar columnas "duplicadas"
print("\nColumnas que contienen 'vote':")
print([col for col in merged.columns if 'vote' in col])

print("\nColumnas que contienen 'popularity':")
print([col for col in merged.columns if 'popularity' in col])

print("\nColumnas que contienen 'runtime':")
print([col for col in merged.columns if 'runtime' in col])

print("\nColumnas que contienen 'revenue':")
print([col for col in merged.columns if 'revenue' in col])

print("\nColumnas que contienen 'original_language':")
print([col for col in merged.columns if 'original_language' in col])

Todas las columnas después del merge:
0: id
1: title
2: vote_average_1M
3: vote_count_1M
4: status
5: release_date_1M
6: revenue_1M
7: runtime_1M
8: adult
9: backdrop_path
10: budget
11: homepage
12: imdb_id
13: original_language_1M
14: original_title_1M
15: overview_1M
16: popularity_1M
17: poster_path
18: tagline_1M
19: genres
20: production_companies
21: production_countries
22: spoken_languages
23: keywords
24: original_language_10k
25: original_title_10k
26: popularity_10k
27: release_date_10k
28: vote_average_10k
29: vote_count_10k
30: genre
31: overview_10k
32: revenue_10k
33: runtime_10k
34: tagline_10k

Columnas que contienen 'vote':
['vote_average_1M', 'vote_count_1M', 'vote_average_10k', 'vote_count_10k']

Columnas que contienen 'popularity':
['popularity_1M', 'popularity_10k']

Columnas que contienen 'runtime':
['runtime_1M', 'runtime_10k']

Columnas que contienen 'revenue':
['revenue_1M', 'revenue_10k']

Columnas que contienen 'original_language':
['original_language_1M', 

In [9]:
merged.info() #resultado del merge

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9952 entries, 0 to 9951
Data columns (total 35 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     9952 non-null   int64  
 1   title                  9952 non-null   object 
 2   vote_average_1M        9952 non-null   float64
 3   vote_count_1M          9952 non-null   int64  
 4   status                 9952 non-null   object 
 5   release_date_1M        9937 non-null   object 
 6   revenue_1M             9952 non-null   int64  
 7   runtime_1M             9952 non-null   int64  
 8   adult                  9952 non-null   bool   
 9   backdrop_path          9677 non-null   object 
 10  budget                 9952 non-null   int64  
 11  homepage               4100 non-null   object 
 12  imdb_id                9845 non-null   object 
 13  original_language_1M   9952 non-null   object 
 14  original_title_1M      9952 non-null   object 
 15  over

In [10]:
# seleccionar variables
movies = merged[
    [
        "original_title_1M",
        "vote_average_1M",        
        "vote_count_1M",         
        "revenue_1M",            
        "budget",                 
        "popularity_1M",       
        "runtime_1M",            
        "genre",                 
        "original_language_1M",   
        "production_countries"    
    ]
]

# renombrar columnas
movies = movies.rename(columns={
    "original_title_1M": "title",
    "vote_average_1M": "vote_average",
    "vote_count_1M": "vote_count", 
    "revenue_1M": "revenue",
    "popularity_1M": "popularity", 
    "runtime_1M": "runtime",
    "original_language_1M": "language",
    "production_countries": "country"
})

In [11]:
movies.info() #resultado del merge

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9952 entries, 0 to 9951
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         9952 non-null   object 
 1   vote_average  9952 non-null   float64
 2   vote_count    9952 non-null   int64  
 3   revenue       9952 non-null   int64  
 4   budget        9952 non-null   int64  
 5   popularity    9952 non-null   float64
 6   runtime       9952 non-null   int64  
 7   genre         9952 non-null   object 
 8   language      9952 non-null   object 
 9   country       9783 non-null   object 
dtypes: float64(2), int64(4), object(4)
memory usage: 777.6+ KB


In [12]:
movies.to_excel('base_combinada.xlsx', index=False)